In [ ]:
!pip install transformers datasets



In [ ]:
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments # Ensure this is imported
)

from datasets import Dataset

In [ ]:
from transformers import DistilBertTokenizerFast
print("OK")


OK


In [ ]:
data = pd.read_csv("phishing_5_percent.csv")

texts = data["body"].astype(str).tolist()
labels = data["label"].tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    X_test,
    truncation=True,
    padding=True,
    max_length=128
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": y_train
})

test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "labels": y_test
})


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
pre_classifier.bias     | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
!pip install -U transformers


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,      # 2 ou 3 suffisent
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    logging_dir="./logs",
    save_strategy="no"
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU est disponible.")
    print(f"Nom du GPU : {torch.cuda.get_device_name(0)}")
else:
    print("GPU n'est pas disponible. Vérifiez les paramètres d'exécution.")

GPU est disponible.
Nom du GPU : Tesla T4


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.075268,0.028932
2,0.009787,0.142677
3,0.007586,0.042716
4,0.000345,0.062952
5,0.000016,0.064295


TrainOutput(global_step=4150, training_loss=0.015356223496316697, metrics={'train_runtime': 767.2195, 'train_samples_per_second': 86.546, 'train_steps_per_second': 5.409, 'total_flos': 2198958817689600.0, 'train_loss': 0.015356223496316697, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(test_dataset)

y_pred = np.argmax(predictions.predictions, axis=1)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[3149    5]
 [  20  146]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3154
           1       0.97      0.88      0.92       166

    accuracy                           0.99      3320
   macro avg       0.98      0.94      0.96      3320
weighted avg       0.99      0.99      0.99      3320

